In [1]:
#importing required libraries
from gensim import corpora
from collections import defaultdict
from gensim import similarities
from gensim import models
import json
import nltk 
from nltk.corpus import stopwords
from googletrans import Translator

In [2]:
#reading json file
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

In [3]:
#append data to json file
def append_data(file, data):
    with open (file, "a", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [4]:
#write json file
def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [5]:
def Translate(data): #List of List
    translator = Translator()
    docs = []
    for article in dat:
        translated_text = translator.translate(article[0],dest='en') #This will change
        #print(translated_text.text)
        #print()
        d=translated_text.text
        docs.append([d,article[1]])  #This too depending on our DB
    return docs #List of List 

In [6]:
#Treat data 
def Treat_Data(docs): #List of List 
    #Creating a list of stopwords
    stoplist = set('for a of the and to in'.split()) 
    #Removing stopwords ( Depends on BD too )
    txts = [[word for word in document[0].lower().split() if word not in stoplist]for document in docs]
    #Calculating frequency of each word 
    frequency = defaultdict(int)
    for text in txts:
        for token in text:
            frequency[token] += 1
    #Removing words that appear only once
    txts = [[token for token in text if frequency[token] > 1]for text in txts]
    #Creating a dictionary
    gensim_dictionary = corpora.Dictionary(txts)
    #Vectorizing the corpus
    gensim_corpus = [gensim_dictionary.doc2bow(text) for text in txts]
    #print(gensim_corpus)
    return gensim_corpus,gensim_dictionary

In [7]:
def LSI(gensim_corpus,gensim_dictionary): #creating LSI model
    lsi = models.LsiModel(gensim_corpus, id2word=gensim_dictionary, num_topics=2)
    return lsi

In [8]:
'''-----Treating the query-------
    Query is the document that will be compared to 
    Doc is an article '''
def Treat_Query(doc):
    #Creating bow vector
    vec_bow = gensim_dictionary.doc2bow(doc.lower().split())
    #Converting the query to LSI space
    vec_lsi = lsi[vec_bow]  
    return vec_lsi
    

In [9]:
def Similarity(vec_lsi,gensim_corpus,lsi):
    index = similarities.MatrixSimilarity(lsi[gensim_corpus])  
    simil = index[vec_lsi]  
    simil=sorted(list(enumerate(simil)),key=lambda item: -item[1])
    #print(simil)
    #print('###########################################################################################')
    return simil


In [10]:
def Match_Sim(docs,simil,nbsim): #Match similar articles with a special id
    for doc_position, doc_score in simil:
        #print("the score is : ",doc_score)
        #print(docs[doc_position][0])  
        #print()
        if doc_score>0.90: 
            docs[doc_position][1] = nbsim 
    return docs

In [11]:
#GENERAL CODE

#This might change with Mysql 
dat = []
nbsim = 0
data = load_data("./dataFR.json")['texts']
for i in range(len(data)):
    dat.append([data[i]["text"],0])
#print(dat)
docs = Translate(dat)
gensim_corpus,gensim_dictionary = Treat_Data(docs)
lsi = LSI(gensim_corpus,gensim_dictionary)
for i in range(len(docs)):
    if docs[i][1] == 0: #Depends on MYSQL
        vec_lsi = Treat_Query(docs[i][0])
        simil = Similarity(vec_lsi,gensim_corpus,lsi)
        nbsim += 1
        docs = Match_Sim(docs,simil,nbsim)
print(docs)

[['The services of the PAF at Houari Boumediene airport in Algiers have succeeded in foiling a new attempt at the illicit transfer of capital. Indeed, they proceeded to seize a large sum in euros, but also in dollars on a passenger who was about to travel to Turkey. Coming from Tizi Ouazou, the traveler in question was about to take his flight to Istanbul. During an ordinary baggage check. Elements of the border police at Algiers airport. They discovered 50,000 euros and 5,000 dollars, carefully concealed in his belongings. According to the Arabic-speaking Ennahar who reported the information, the PAF arrested this passenger. And the seizure of this large sum. An investigation was launched and the accused was presented on Wednesday, August 31, before the court of Dar El Beida, located in Algiers. And this, for violation of the regulations governing the transfer of capital from and to Algeria. During his hearings, the traveler from Tizi Ouazou declared that this money. Was destined for 